In [1]:
import pandas as pd
import numpy as np

In [2]:
f = lambda x: (x.replace(",","")) 
df = pd.read_csv("data/raw_data.tsv", delimiter="\t", converters = {"기초금액":f, "예정가격":f, "1순위가격":f, "참여업체수":f})
print (f"Rows:{len(df)}")

Rows:11617


In [3]:
df.head()

,번호,공사명,종목,발주처,예가범위,지역제한,기초금액,예정가격,예정가격사정률,1순위가격,1순위사정률,1순위업체,참여업체수,입찰일
0,2202,진도읍시가지 LED가로등 교체 및 신설공사 [20200611936-00],전기,전라남도 진도군,103% ~ 97%,진도,39563000,39507475,99.85965,34694492,99.94220,(주)삼강전력,21,2020/06/12 15:00
1,2201,6070 낭만곡성 청춘어람센터 신축 전기공사 [20200611794-00],전기,전라남도 곡성군,103% ~ 97%,곡성,83160000,82069400,98.68855,72289900,99.06968,오에스종합건설 주식회사,23,2020/06/12 15:00
2,2200,옥과면 공영주차장 조성 전기공사 [20200611788-00],전기,전라남도 곡성군,103% ~ 97%,곡성,25355000,25599250,100.96332,22464150,100.97270,(주)회성전력,23,2020/06/12 15:00
3,2199,곡성읍 공영주차장 조성 전기공사 [20200611781-00],전기,전라남도 곡성군,103% ~ 97%,곡성,36454000,36215375,99.34540,31803760,99.42850,주식회사 대국전력,23,2020/06/12 15:00
4,2198,팔마대교~풍덕교 사이 스윙교 설치 전기공사 [20200611713-00],전기,전라남도 순천시,103% ~ 97%,순천,36993000,36880025,99.69460,32365313,99.70980,주식회사서정이엔씨,114,2020/06/12 13:00


In [4]:
df['기초금액'] = df['기초금액'].astype(int)
df['예정가격'] = df['예정가격'].astype(int)
df['참여업체수'] = df['참여업체수'].astype(int)
df = df[df['기초금액'] != 0]
df = df[df['예정가격'] != 0]
df = df[df['참여업체수'] != 0]
# df['1순위가격'] = df['1순위가격'].astype(int)
#df.describe()

In [5]:
df['공사명'] = df['공사명'].str.replace("\[","")
df['공사명'] = df['공사명'].str.replace("\]","")
df['공사명'] = df['공사명'].str.replace('[0-9]', '')
df['공사명'] = df['공사명'].str.replace('-', '')

df.head()

,번호,공사명,종목,발주처,예가범위,지역제한,기초금액,예정가격,예정가격사정률,1순위가격,1순위사정률,1순위업체,참여업체수,입찰일
0,2202,진도읍시가지 LED가로등 교체 및 신설공사,전기,전라남도 진도군,103% ~ 97%,진도,39563000,39507475,99.85965,34694492,99.94220,(주)삼강전력,21,2020/06/12 15:00
1,2201,낭만곡성 청춘어람센터 신축 전기공사,전기,전라남도 곡성군,103% ~ 97%,곡성,83160000,82069400,98.68855,72289900,99.06968,오에스종합건설 주식회사,23,2020/06/12 15:00
2,2200,옥과면 공영주차장 조성 전기공사,전기,전라남도 곡성군,103% ~ 97%,곡성,25355000,25599250,100.96332,22464150,100.97270,(주)회성전력,23,2020/06/12 15:00
3,2199,곡성읍 공영주차장 조성 전기공사,전기,전라남도 곡성군,103% ~ 97%,곡성,36454000,36215375,99.34540,31803760,99.42850,주식회사 대국전력,23,2020/06/12 15:00
4,2198,팔마대교~풍덕교 사이 스윙교 설치 전기공사,전기,전라남도 순천시,103% ~ 97%,순천,36993000,36880025,99.69460,32365313,99.70980,주식회사서정이엔씨,114,2020/06/12 13:00


In [6]:
df['bidding_month'] = pd.DatetimeIndex(df['입찰일']).month

In [7]:
df['bidding_quarter'] = '1Q'
df.loc[df['bidding_month'] < 4, 'bidding_quarter'] = '1Q'
df.loc[(df['bidding_month'] >= 4) & (df['bidding_month'] < 7), 'bidding_quarter'] = '2Q'
df.loc[(df['bidding_month'] >= 7) & (df['bidding_month'] < 10), 'bidding_quarter'] = '3Q'
df.loc[(df['bidding_month'] >= 10) & (df['bidding_month'] <= 12), 'bidding_quarter'] = '4Q'

In [8]:
df = df[df['참여업체수'] > 20]



In [9]:
def get_bidding_count(project_owner, bidding_date, df):

    return len(df[(df["발주처"] == project_owner) & (df['입찰일'] < bidding_date)])


def get_latest_target_rate(project_owner, bidding_date, df):
    
    temp = df[(df["발주처"] == project_owner) & (df['입찰일'] < bidding_date)]
    
    if len(temp) > 0 :
        return temp['예정가격사정률'].max(), temp['예정가격사정률'].min(), temp['예정가격사정률'].mean(), temp['예정가격사정률'].std()
    else :
        return 0, 0, 0, 0

In [10]:
df2 = df.copy()
df[['max_target_rate', 'min_target_rate', 'mean_target_rate', 'std_target_rate']] = df.apply(lambda x:get_latest_target_rate(x['발주처'], x['입찰일'], df2), axis=1, result_type='expand')
df['count_bidding'] = df.apply(lambda x:get_bidding_count(x['발주처'], x['입찰일'], df2), axis=1)

In [11]:
out = df.copy()

In [12]:
out = out[out['mean_target_rate'] != 0]

In [13]:
out['mean_target_rate'].min()

95.35399

In [14]:
len(out)

9841

In [15]:
out.head()

,번호,공사명,종목,발주처,예가범위,지역제한,기초금액,예정가격,예정가격사정률,1순위가격,...,1순위업체,참여업체수,입찰일,bidding_month,bidding_quarter,max_target_rate,min_target_rate,mean_target_rate,std_target_rate,count_bidding
0,2202,진도읍시가지 LED가로등 교체 및 신설공사,전기,전라남도 진도군,103% ~ 97%,진도,39563000,39507475,99.85965,34694492,...,(주)삼강전력,21,2020/06/12 15:00,6,2Q,101.64736,98.16086,100.176866,0.722864,42
1,2201,낭만곡성 청춘어람센터 신축 전기공사,전기,전라남도 곡성군,103% ~ 97%,곡성,83160000,82069400,98.68855,72289900,...,오에스종합건설 주식회사,23,2020/06/12 15:00,6,2Q,101.08701,98.10806,99.888283,0.706660,72
2,2200,옥과면 공영주차장 조성 전기공사,전기,전라남도 곡성군,103% ~ 97%,곡성,25355000,25599250,100.96332,22464150,...,(주)회성전력,23,2020/06/12 15:00,6,2Q,101.08701,98.10806,99.888283,0.706660,72
3,2199,곡성읍 공영주차장 조성 전기공사,전기,전라남도 곡성군,103% ~ 97%,곡성,36454000,36215375,99.34540,31803760,...,주식회사 대국전력,23,2020/06/12 15:00,6,2Q,101.08701,98.10806,99.888283,0.706660,72
4,2198,팔마대교~풍덕교 사이 스윙교 설치 전기공사,전기,전라남도 순천시,103% ~ 97%,순천,36993000,36880025,99.69460,32365313,...,주식회사서정이엔씨,114,2020/06/12 13:00,6,2Q,101.69255,98.03514,99.839237,0.704000,288


In [16]:
out2 = out[['공사명', '종목', '발주처', '지역제한', '기초금액', '예정가격', '입찰일', '참여업체수', 'bidding_month', 'bidding_quarter','max_target_rate','min_target_rate', 'mean_target_rate', 'std_target_rate', 'count_bidding', '예정가격사정률', '1순위사정률']]
out2.columns = ['project_name', 'category', 'project_owner', 'restriction', 'basic_price', 'target_price', 'bidding_date', 'competitors', 'bidding_month','bidding_quarter', 'max_target_rate','min_target_rate', 'mean_target_rate', 'std_target_rate', 'count_bidding', 'target_rate', 'winner_rate']
out2.head()

,project_name,category,project_owner,restriction,basic_price,target_price,bidding_date,competitors,bidding_month,bidding_quarter,max_target_rate,min_target_rate,mean_target_rate,std_target_rate,count_bidding,target_rate,winner_rate
0,진도읍시가지 LED가로등 교체 및 신설공사,전기,전라남도 진도군,진도,39563000,39507475,2020/06/12 15:00,21,6,2Q,101.64736,98.16086,100.176866,0.722864,42,99.85965,99.94220
1,낭만곡성 청춘어람센터 신축 전기공사,전기,전라남도 곡성군,곡성,83160000,82069400,2020/06/12 15:00,23,6,2Q,101.08701,98.10806,99.888283,0.706660,72,98.68855,99.06968
2,옥과면 공영주차장 조성 전기공사,전기,전라남도 곡성군,곡성,25355000,25599250,2020/06/12 15:00,23,6,2Q,101.08701,98.10806,99.888283,0.706660,72,100.96332,100.97270
3,곡성읍 공영주차장 조성 전기공사,전기,전라남도 곡성군,곡성,36454000,36215375,2020/06/12 15:00,23,6,2Q,101.08701,98.10806,99.888283,0.706660,72,99.34540,99.42850
4,팔마대교~풍덕교 사이 스윙교 설치 전기공사,전기,전라남도 순천시,순천,36993000,36880025,2020/06/12 13:00,114,6,2Q,101.69255,98.03514,99.839237,0.704000,288,99.69460,99.70980


In [135]:
out = out[['발주처', '지역제한', '기초금액', '참여업체수', 'bidding_month', 'bidding_quarter','max_target_rate','min_target_rate', 'mean_target_rate', 'std_target_rate', 'count_bidding', '예정가격사정률', '1순위사정률']]
out.columns = ['project_owner', 'restriction', 'basic_price', 'competitors', 'bidding_month','bidding_quarter', 'max_target_rate','min_target_rate', 'mean_target_rate', 'std_target_rate', 'count_bidding', 'target_rate', 'winner_rate']
out.head()

,project_owner,restriction,basic_price,competitors,bidding_month,bidding_quarter,max_target_rate,min_target_rate,mean_target_rate,std_target_rate,count_bidding,target_rate,winner_rate
0,전라남도 진도군,진도,39563000,21,6,2Q,101.64736,98.16086,100.176866,0.722864,42,99.85965,99.94220
1,전라남도 곡성군,곡성,83160000,23,6,2Q,101.08701,98.10806,99.888283,0.706660,72,98.68855,99.06968
2,전라남도 곡성군,곡성,25355000,23,6,2Q,101.08701,98.10806,99.888283,0.706660,72,100.96332,100.97270
3,전라남도 곡성군,곡성,36454000,23,6,2Q,101.08701,98.10806,99.888283,0.706660,72,99.34540,99.42850
4,전라남도 순천시,순천,36993000,114,6,2Q,101.69255,98.03514,99.839237,0.704000,288,99.69460,99.70980


In [136]:
mask = np.random.rand(len(out)) < 0.95
train = out[mask]
test  = out[~mask]

In [137]:
train.to_csv("data/train_0713_5.csv", encoding="utf-8")
test.to_csv("data/test_0713_5.csv", encoding="utf-8")

In [17]:
out2.to_csv("data/project_list.csv", encoding="utf-8")